In [ ]:
from pathlib import Path
import json
import re
import unittest
from unittest import mock
import pandas as pd
import trustyai as trustyai
from platform import python_version
from trustyai.metrics.fairness.group import statistical_parity_difference
from trustyai.model import output
from pandas.testing import assert_frame_equal
import numpy as np
import scipy
import sklearn
from scipy import special
from scipy import integrate
from sklearn import datasets
from sklearn.model_selection import train_test_split
import matplotlib
import matplotlib.pyplot as plt
import kafka
from kafka import KafkaConsumer, KafkaProducer, TopicPartition
from kafka.producer.buffer import SimpleBufferPool
from kafka import KafkaConsumer
from kafka.errors import KafkaConfigurationError
import boto3
import jupyterlab as jp
import nbdime
import nbgitpuller

def get_major_minor(s):
    return '.'.join(s.split('.')[:2])

def load_expected_versions() -> dict:
    lock_file = Path('./expected_versions.json')
    data = {}

    with open(lock_file, 'r') as file:
        data = json.load(file)

    return data    

def get_expected_version(dependency_name: str) -> str:
    raw_value = expected_versions.get(dependency_name)
    raw_version = re.sub(r'^\D+', '', raw_value)
    return get_major_minor(raw_version) 

class TestPythonVersion(unittest.TestCase):
    def test_version(self):
        expected_major_minor = get_expected_version('Python')
        actual_major_minor = get_major_minor(python_version())
        self.assertEqual(actual_major_minor, expected_major_minor, "incorrect version")

class TestTrustyaiNotebook(unittest.TestCase):

    def test_trustyai_version(self):
        expected_major_minor = get_expected_version('TrustyAI')
        actual_major_minor = get_major_minor(trustyai.__version__)
        self.assertEqual(actual_major_minor, expected_major_minor, "incorrect version")

    def test_fairnessmetrics(self):
        url_unbiased = "https://raw.githubusercontent.com/opendatahub-io/notebooks/main/jupyter/trustyai/ubi9-python-3.11/test/income-unbiased.csv"
        nobias = pd.read_csv(url_unbiased, index_col=False)

        nobias = pd.read_csv(url_unbiased, index_col=False)
        nobias.groupby(['gender', 'income'])['income'].count()
        nobias.groupby(['gender', 'income'])['income'].count().unstack().plot.bar()

        nobias_privileged = nobias[nobias.gender == 1]
        nobias_unprivileged = nobias[nobias.gender == 0]
        favorable = output("income", dtype="number", value=1)
        score = statistical_parity_difference(privileged=nobias_privileged,
                                              unprivileged=nobias_unprivileged,
                                              favorable=[favorable])
        self.assertTrue(score >= 0.0036255104824703954)
        print("On the test_fairness_metrics test case the statistical_parity_difference score for this dataset is between the threshold [-0.1,0.1], which classifies the model as reasonably fair.")

    def test_datafairness(self):
        url_biased = "https://raw.githubusercontent.com/opendatahub-io/notebooks/main/jupyter/trustyai/ubi9-python-3.11/test/income-biased.csv"
        bias = pd.read_csv(url_biased, index_col=False)

        # Perform the data manipulations
        grouped_counts = bias.groupby(['gender', 'income'])['income'].count()
        unstacked_counts = bias.groupby(['gender', 'income'])['income'].count().unstack()
        unstacked_counts.plot.bar()

        bias_privileged = bias[bias.gender == 1]
        bias_unprivileged = bias[bias.gender == 0]
        favorable = output("income", dtype="number", value=1)
        score = statistical_parity_difference(privileged=bias_privileged,
                                              unprivileged=bias_unprivileged,
                                              favorable=[favorable])
        self.assertTrue(score <= -0.15670061634672994)
        print("On the test_bias_metrics test case the statistical_parity_difference score for this dataset, as expected, is outside the threshold [-0.1,0.1], which classifies the model as unfair.")

expected_versions = load_expected_versions()
unittest.main(argv=[''], verbosity=2, exit=False)